In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Schaltkreise erstellen

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen, diese oder neuere Versionen zu verwenden.

```
qiskit[all]~=2.3.0
```
</details>
Diese Seite wirft einen genaueren Blick auf die [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) Klasse im Qiskit SDK, einschließlich einiger fortgeschrittener Methoden, die Sie zum Erstellen von Quantenschaltkreisen verwenden können.
## Was ist ein Quantenschaltkreis?
Ein einfacher Quantenschaltkreis ist eine Sammlung von Qubits und eine Liste von Instruktionen, die auf diese Qubits wirken. Zur Demonstration erstellt die folgende Zelle einen neuen Schaltkreis mit zwei neuen Qubits und zeigt dann das [`qubits`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qubits) Attribut des Schaltkreises an, welches eine Liste von [`Qubits`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Qubit) in Reihenfolge vom niederwertigsten Bit $q_0$ bis zum höchstwertigen Bit $q_n$ ist.

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.qubits

[<Qubit register=(2, "q"), index=0>, <Qubit register=(2, "q"), index=1>]

Multiple `QuantumRegister` and `ClassicalRegister` objects can be combined to create a circuit. Every [`QuantumRegister`](/docs/api/qiskit/circuit#qiskit.circuit.QuantumRegister) and [`ClassicalRegister`](/docs/api/qiskit/circuit#qiskit.circuit.ClassicalRegister) can also be named.

In [2]:
from qiskit.circuit import QuantumRegister, ClassicalRegister

qr1 = QuantumRegister(2, "qreg1")  # Create a QuantumRegister with 2 qubits
qr2 = QuantumRegister(1, "qreg2")  # Create a QuantumRegister with 1 qubit
cr1 = ClassicalRegister(3, "creg1")  # Create a ClassicalRegister with 3 cbits

combined_circ = QuantumCircuit(
    qr1, qr2, cr1
)  # Create a quantum circuit with 2 QuantumRegisters and 1 ClassicalRegister
combined_circ.qubits

[<Qubit register=(2, "qreg1"), index=0>,
 <Qubit register=(2, "qreg1"), index=1>,
 <Qubit register=(1, "qreg2"), index=0>]

Mehrere `QuantumRegister` und `ClassicalRegister` Objekte können kombiniert werden, um einen Schaltkreis zu erstellen. Jedes [`QuantumRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.QuantumRegister) und [`ClassicalRegister`](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.ClassicalRegister) kann auch benannt werden.

In [3]:
desired_qubit = qr2[0]  # Qubit 0 of register 'qreg2'

print("Index:", combined_circ.find_bit(desired_qubit).index)
print("Register:", combined_circ.find_bit(desired_qubit).registers)

Index: 2
Register: [(QuantumRegister(1, 'qreg2'), 0)]


Adding an instruction to the circuit appends the instruction to the circuit's [`data`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#data) attribute. The following cell output shows `data` is a list of [`CircuitInstruction`](/docs/api/qiskit/qiskit.circuit.CircuitInstruction) objects, each of which has an `operation` attribute, and a `qubits` attribute.

In [4]:
qc.x(0)  # Add X-gate to qubit 0
qc.data

[CircuitInstruction(operation=Instruction(name='x', num_qubits=1, num_clbits=0, params=[]), qubits=(<Qubit register=(2, "q"), index=0>,), clbits=())]

Sie können den Index und das Register eines Qubits finden, indem Sie die [`find_bit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.find_bit) Methode des Schaltkreises und deren Attribute verwenden.

In [5]:
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg" alt="Output of the previous code cell" />

Circuit instruction objects can contain "definition" circuits that describe the instruction in terms of more fundamental instructions. For example, the [X-gate](/docs/api/qiskit/qiskit.circuit.library.XGate) is defined as a specific case of the [U3-gate](/docs/api/qiskit/qiskit.circuit.library.U3Gate), a more general single-qubit gate.

In [6]:
# Draw definition circuit of 0th instruction in `qc`
qc.data[0].operation.definition.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg" alt="Output of the previous code cell" />

Das Hinzufügen einer Instruktion zum Schaltkreis hängt die Instruktion an das [`data`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#data) Attribut des Schaltkreises an. Die folgende Zellenausgabe zeigt, dass `data` eine Liste von [`CircuitInstruction`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.CircuitInstruction) Objekten ist, von denen jedes ein `operation` Attribut und ein `qubits` Attribut hat.

In [7]:
from qiskit.circuit.library import HGate

qc = QuantumCircuit(1)
qc.append(
    HGate(),  # New HGate instruction
    [0],  # Apply to qubit 0
)
qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg" alt="Output of the previous code cell" />

To combine two circuits, use the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method. This accepts another [`QuantumCircuit`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit) and an optional list of qubit mappings.

<Admonition type="note">
    The [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method returns a new circuit and does **not** mutate either circuit it acts on. To mutate the circuit on which you're calling the [`compose`](/docs/api/qiskit/qiskit.circuit.QuantumCircuit#compose) method, use the argument `inplace=True`.
</Admonition>

In [8]:
qc_a = QuantumCircuit(4)
qc_a.x(0)

qc_b = QuantumCircuit(2, name="qc_b")
qc_b.y(0)
qc_b.z(1)

# compose qubits (0, 1) of qc_a to qubits (1, 3) of qc_b respectively
combined = qc_a.compose(qc_b, qubits=[1, 3])
combined.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/29152dfa-2275-4bc4-aadb-82185b9e0e86-0.svg" alt="Output of the previous code cell" />

Der einfachste Weg, diese Information zu betrachten, ist über die [`draw`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#draw) Methode, die eine Visualisierung eines Schaltkreises zurückgibt. Siehe [Schaltkreise visualisieren](./visualize-circuits) für verschiedene Möglichkeiten zur Darstellung von Quantenschaltkreisen.

In [9]:
inst = qc_b.to_instruction()
qc_a.append(inst, [1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/81b682dd-45cb-4492-809e-d9e8ebbf5600-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/43a57258-3e33-4071-8a48-2bf127c8a5be-0.svg)

Schaltkreis-Instruktionsobjekte können "Definitions"-Schaltkreise enthalten, die die Instruktion in Bezug auf fundamentalere Instruktionen beschreiben. Beispielsweise ist das [X-Gate](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.XGate) als ein spezieller Fall des [U3-Gates](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.U3Gate) definiert, einem allgemeineren Single-Qubit-Gate.

In [10]:
gate = qc_b.to_gate().control()
qc_a.append(gate, [0, 1, 3])
qc_a.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/653e2427-e301-4d2f-84de-1959185ace8e-0.svg)

Instruktionen und Schaltkreise sind insofern ähnlich, als dass beide Operationen auf Bits und Qubits beschreiben, aber sie haben unterschiedliche Zwecke:

- Instruktionen werden als fest behandelt, und ihre Methoden geben normalerweise neue Instruktionen zurück (ohne das ursprüngliche Objekt zu verändern).
- Schaltkreise sind so konzipiert, dass sie über viele Codezeilen hinweg aufgebaut werden, und [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) Methoden verändern oft das bestehende Objekt.
### Was ist die Schaltkreistiefe?
Die [depth()](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.depth) eines Quantenschaltkreises ist ein Maß für die Anzahl der "Schichten" von Quantengattern, die parallel ausgeführt werden, um die durch den Schaltkreis definierte Berechnung abzuschließen. Da Quantengatter Zeit zur Implementierung benötigen, entspricht die Tiefe eines Schaltkreises ungefähr der Zeit, die der Quantencomputer zur Ausführung des Schaltkreises benötigt. Daher ist die Tiefe eines Schaltkreises eine wichtige Größe, die verwendet wird, um zu messen, ob ein Quantenschaltkreis auf einem Gerät ausgeführt werden kann.

Der Rest dieser Seite veranschaulicht, wie Quantenschaltkreise manipuliert werden können.
## Schaltkreise erstellen
Methoden wie [`QuantumCircuit.h`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#h) und [`QuantumCircuit.cx`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#cx) fügen spezifische Instruktionen zu Schaltkreisen hinzu. Um Instruktionen allgemeiner zu einem Schaltkreis hinzuzufügen, verwenden Sie die [`append`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#append) Methode. Diese nimmt eine Instruktion und eine Liste von Qubits, auf die die Instruktion angewendet werden soll. Siehe die [Circuit Library API Dokumentation](https://docs.quantum.ibm.com/api/qiskit/circuit_library) für eine Liste unterstützter Instruktionen.

In [11]:
qc_a.decompose().draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/66813cae-9841-47ea-96b7-8fd7b82e9759-0.svg)

Um zwei Schaltkreise zu kombinieren, verwenden Sie die [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose) Methode. Diese akzeptiert einen anderen [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) und eine optionale Liste von Qubit-Zuordnungen.

> **Note:** Die [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose) Methode gibt einen neuen Schaltkreis zurück und verändert **nicht** einen der beiden Schaltkreise, auf die sie wirkt. Um den Schaltkreis zu verändern, auf dem Sie die [`compose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#compose) Methode aufrufen, verwenden Sie das Argument `inplace=True`.

In [12]:
qc1 = QuantumCircuit(2, 2)
qc1.measure(0, 1)
qc1.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/0cdb2273-0.svg" alt="Output of the previous code cell" />

In [13]:
qc2 = QuantumCircuit(2)
qc2.measure_all()
qc2.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/6f33698c-0.svg" alt="Output of the previous code cell" />

In [14]:
qc3 = QuantumCircuit(2)
qc3.x(1)
qc3.measure_active()
qc3.draw("mpl", cregbundle=False)

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ed362e64-d6a4-4dfd-a5cf-5e6bdc7a81b5-0.svg)

Um zu sehen, was vor sich geht, können Sie die [`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose) Methode verwenden, um jede Instruktion in ihre Definition zu erweitern.

> **Note:** Die [`decompose`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#decompose) Methode gibt einen neuen Schaltkreis zurück und verändert **nicht** den Schaltkreis, auf den sie wirkt.

In [15]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit import Parameter

angle = Parameter("angle")  # undefined number

# Create and optimize circuit once
qc = QuantumCircuit(1)
qc.rx(angle, 0)
qc = generate_preset_pass_manager(
    optimization_level=3, basis_gates=["u", "cx"]
).run(qc)

qc.draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/a580552c-d585-4047-99f0-32aafd06e4f3-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/3c0633db-929b-4428-a888-7a3d493bd6dd-0.svg)

<span id="measure-qubits"></span>
## Qubits messen
Messungen werden verwendet, um die Zustände einzelner Qubits zu sampeln und die Ergebnisse in ein klassisches Register zu übertragen. Beachten Sie, dass, wenn Sie Schaltkreise an ein [Sampler](./primitives#sampler) Primitive übermitteln, Messungen erforderlich sind. Schaltkreise, die an ein [Estimator](./primitives#estimator) Primitive übermittelt werden, dürfen jedoch keine Messungen enthalten.

Qubits können mit drei Methoden gemessen werden: [`measure`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#qiskit.circuit.QuantumCircuit.measure), [`measure_all`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_all) und [`measure_active`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#measure_active). Um zu erfahren, wie gemessene Ergebnisse visualisiert werden können, siehe die Seite [Ergebnisse visualisieren](./visualize-results).

1. `QuantumCircuit.measure`: misst jedes Qubit im ersten Argument auf das klassische Bit, das als zweites Argument angegeben ist. Diese Methode ermöglicht volle Kontrolle darüber, wo das Messergebnis gespeichert wird.

2. `QuantumCircuit.measure_all`: nimmt kein Argument und kann für Quantenschaltkreise ohne vordefinierte klassische Bits verwendet werden. Sie erstellt klassische Drähte und speichert Messergebnisse in Reihenfolge. Zum Beispiel wird die Messung von Qubit $q_i$ in cbit $meas_i$ gespeichert. Sie fügt auch eine Barriere vor der Messung hinzu.

3. `QuantumCircuit.measure_active`: ähnlich wie `measure_all`, misst aber nur Qubits, die Operationen haben.

In [16]:
circuits = []
for value in range(100):
    circuits.append(qc.assign_parameters({angle: value}))

circuits[0].draw("mpl")

<Image src="../docs/images/guides/construct-circuits/extracted-outputs/85af6231-921a-4130-99d3-f6998f761df8-0.svg" alt="Output of the previous code cell" />

You can find a list of a circuit's undefined parameters in its `parameters` attribute.

In [17]:
qc.parameters

ParameterView([Parameter(angle)])

### Change a parameter's name

By default, parameter names for a parameterized circuit are prefixed by `x`- for example, `x[0]`. You can change the names after they are defined, as shown in the following example.

In [18]:
from qiskit.circuit.library import z_feature_map
from qiskit.circuit import ParameterVector

# Define a parameterized circuit with default names
# For example, x[0]
circuit = z_feature_map(2)

# Set new parameter names
# They will now be prefixed by `hi` instead
# For example, hi[0]
training_params = ParameterVector("hi", 2)

# Assign parameter names to the quantum circuit
circuit = circuit.assign_parameters(parameters=training_params)

![Output of the previous code cell](../docs/images/guides/construct-circuits/extracted-outputs/ca3f225f-0.svg)

## Parametrisierte Schaltkreise

Viele kurzfristige Quantenalgorithmen beinhalten die Ausführung vieler Variationen eines Quantenschaltkreises. Da das Konstruieren und Optimieren großer Schaltkreise rechenintensiv sein kann, unterstützt Qiskit **parametrisierte** Schaltkreise. Diese Schaltkreise haben undefinierte Parameter, und ihre Werte müssen erst kurz vor der Ausführung des Schaltkreises definiert werden. Dies ermöglicht es Ihnen, die Schaltkreiskonstruktion und -optimierung aus der Hauptprogrammschleife zu verschieben. Die folgende Zelle erstellt und zeigt einen parametrisierten Schaltkreis an.